In [1]:
import pandas as pd
import datetime
import glob

In [2]:
def read_in_data(location, path):
    files = glob.glob(path + location + "_agg/*")
    df = pd.read_csv(files[0])
    filename = files[0].split("agg")[-1][1:-4]
    df["file"] = filename
    for file in files[1:]:
        new_df = pd.read_csv(file)
        filename = file.split("agg")[-1][1:-4]
        new_df["file"] = filename
        df = df.append(new_df)
    return df

In [3]:
df_wrist = read_in_data("wrist", '')
df_hip = read_in_data("hip", '')

In [4]:
DO_log_final_df = pd.read_csv('DO_log_final.csv')

,id,obs,type,start_month,start_day,start_year,start_time,stop_time,duration,session
1,AM01,DO1,H,10,6,2017,16:43:57,18:45:00,2:01:03,2
2,AM02,DO1,H,7,24,2017,13:17:10,15:17:32,2:00:22,1
6,AM03,DO2,H,7,27,2017,13:01:29,15:00:39,1:59:10,2
12,AM07,DO1,H,8,2,2017,9:01:18,11:01:27,2:00:09,1
40,AM19,DO1,H,11,9,2017,11:33:03,13:34:00,2:00:57,1


In [7]:
def datetime_filter(location, data, log):
    log = log[log['type'] == location[0].upper()]
    
    log['date'] = pd.to_datetime(log[['start_day','start_month','start_year']]
                   .astype(str).apply(' '.join, 1), format='%d %m %Y').astype(str)
    log['start_time'] = pd.to_datetime(log['start_time'], format = '%H:%M:%S').apply(lambda x: x.time())
    log['stop_time'] = pd.to_datetime(log['stop_time'], format = '%H:%M:%S').apply(lambda x: x.time())
    
    data['actual_datetime'] = pd.to_datetime(data['actual_datetime'], format = '%Y-%m-%d %H:%M:%S')
    data['actual_date'] = data['actual_datetime'].apply(lambda x: x.date()).astype(str)
    data['actual_time'] = data['actual_datetime'].apply(lambda x: x.time())
    
    joined = pd.merge(data, log, how = 'left', left_on = 'actual_date', right_on = 'date')
    
    joined = joined[(joined['actual_time'] >= joined['start_time']) & 
                    (joined['actual_time'] <= joined['stop_time'])]
    
    return joined[['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 'actual_datetime', 'file']]

In [8]:
hip_filtered = datetime_filter('hip', df_hip, DO_log_final_df)
wrist_filtered = datetime_filter('wrist', df_wrist, DO_log_final_df)

<ipython-input-7-392f920baa60>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['date'] = pd.to_datetime(log[['start_day','start_month','start_year']]
<ipython-input-7-392f920baa60>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['start_time'] = pd.to_datetime(log['start_time'], format = '%H:%M:%S').apply(lambda x: x.time())
<ipython-input-7-392f920baa60>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [9]:
hip_filtered['actual_datetime'].apply(lambda x: x.date()).unique()

array([datetime.date(2017, 10, 6), datetime.date(2017, 7, 24),
       datetime.date(2017, 7, 27), datetime.date(2017, 8, 2),
       datetime.date(2017, 11, 9), datetime.date(2017, 12, 2),
       datetime.date(2018, 2, 17), datetime.date(2018, 2, 24),
       datetime.date(2018, 5, 18)], dtype=object)

In [10]:
DO_log_final_df[DO_log_final_df['type'] == 'H'][['start_month', 'start_day', 'start_year']]

,start_month,start_day,start_year
1,10,6,2017
2,7,24,2017
6,7,27,2017
12,8,2,2017
40,11,9,2017
42,12,2,2017
50,2,17,2018
53,2,24,2018
54,2,24,2018
56,5,18,2018


In [11]:
hip_filtered.to_csv('hip_filtered.csv', index = False)
wrist_filtered.to_csv('wrist_filtered.csv', index = False)